In [5]:
import pandas as pd
import codecs 
import re

In [2]:
[x for x in "सक्दैनः"]

['स', 'क', '्', 'द', 'ै', 'न', 'ः']

In [3]:
ratopati = pd.read_csv('./ratopati/ratopati_news_final.csv')
ujyaalo = pd.read_csv('./ujyaaloonline/ujyaalo_news_final.csv')
techpana = pd.read_csv('./techpana/techpana_news_final.csv')
nepalikhabar = pd.read_csv('./nepalikhabar/nepalikhabar_news_final.csv')
ekagaj = pd.read_csv('./ekagaj/ekagaj_news_final.csv')
annapurna = pd.read_csv('./annapurna/annapurna_news_final.csv')
dcnepal = pd.read_csv('./dcnepal/dcnepal_news_final.csv')
ekantipur = pd.read_csv('./ekantipur/ekantipur_news_final.csv')
gorkhapatra = pd.read_csv('./gorkhapatra/gorkhapatra_news_final.csv')
imagekhabar = pd.read_csv('./imagekhabar/imagekhabar_news_final.csv')
nagarik = pd.read_csv('./nagarik/nagarik_news_final.csv')

आरोहणका लागि अनुमति नभएका हिमाल खुला गर्नुपर्छ : राष्ट्रपति<br>
जनतामा छाएको निराशालाई सम्बोधन गर्न सबै दलहरु एक हुनुपर्छ : लामिछाने<br>
अर्थतन्त्रको दबाब हटाउन राष्ट्रिय गोलमेच गर्छौँ : प्रधानमन्त्री दाहाल<br>
बजेट डडेलधुरा, गोरखा र नुवाकोट केन्द्रित भयो भन्ने सुनेको छु जवाफ माग्छुः माधव नेपाल<br>
उच्चस्तरीय आयोग गठनबारे विश्वप्रकाशको टिप्पणीः प्रतिपक्षलाई विश्वासमा लिन बनाइएको रंगिन टालो<br>


Seperate at (" : ") and remove whichever is the shortest.<br>
For last if both are >=4 remove such article.<br>
(Same as above) Also remove any article such that (" : ") exists and both of the side has >2 words and the differnece between the words in both sides is -2 <= = <= 2<br>

पर्यटन बोर्डले बजेट सुधार नगरेसम्म बजेट पास हुन सक्दैनः मन्त्री किराँती<br>


Seperate at (" : "), if index 0 length < index 1 length, keep the index 0, just remove the colon given it doesnot satisfy the >=4 constraints<br>
प्रदीप पौडेलको खबरदारी : सरकारले सार्वजनिक जग्गाको दुरुपयोग नगरोस्


First try checking manually if feasible<br>
राष्ट्रियसभाका बैठक : सांसदको सङ्ख्या घटाउन माग

In [4]:
'''
1. Remove first part
2. Remove second part
3. Remove third part
4. Remove first, second part
5. Remove first, third part
6. Remove second, third part
7. Remove all
8. Remove None
'''

def resolve_slicing(title, join=" ", split=" : "):
    print(title)
    # title_split = title.split(split)
    # operation = input("Enter command: ")
    # if operation == "1":
    #     return title_split[1] + join + title_split[2]
    # elif operation == "2":
    #     return title_split[0] + join + title_split[2]
    # elif operation == "3":
    #     return title_split[0] + join + title_split[1] + join.strip()
    # elif operation == "4":
    #     return title_split[2]
    # elif operation == "5":
    #     return title_split[1] + join.strip()
    # elif operation == "6":
    #     return title_split[0] + join.strip()
    # elif operation == "7":
    #     return ""
    # elif operation == "8":
    #     return join.join(title_split)
    
def predefined_slicing_nagarik(title, join=" ", split=" : "):

    if title == "नगरपालिकामा 'एक शिक्षक : एक ल्यापटप' कार्यक्रम: शिक्षक भए खुसी!":
        return "नगरपालिकामा 'एक शिक्षक एक ल्यापटप' कार्यक्रम शिक्षक भए खुसी"
    
    elif title == "लम्पिस्किन रोगको संक्रमण क्रमशः घटने क्रममा छः मन्त्री भुसाल":
        return "लम्पिस्किन रोगको संक्रमण घटने क्रममा छ"
    
    elif title == "अमेरिका निर्वाचन २०२०ः जोर्जियामा हातैले पुनः मतगणना गरिने":
        return "अमेरिका निर्वाचन २०२० जोर्जियामा हातैले पुन मतगणना गरिने"
    
    elif title == "नेपालमा राजतन्त्र पुनः स्थापना हुन असम्भव छः संयोजक बास्तोला":
        return "नेपालमा राजतन्त्र पुन स्थापना हुन असम्भव छ"
    
    elif title == "रास्वपालाई पुन: सरकारमा ल्याउने विषयमा छलफल भयाे: अध्यक्ष लिङ्देन":
        return "रास्वपालाई पुन सरकारमा ल्याउने विषयमा छलफल भयाे"
    
    elif title == "राष्ट्रिय स्वतन्त्र पार्टीलाई पुनः सरकारमा ल्याउने प्रयत्न जारी छः मन्त्री शर्मा":
        return "राष्ट्रिय स्वतन्त्र पार्टीलाई पुन सरकारमा ल्याउने प्रयत्न जारी छ"
    
    elif title == "आयोग कडा भएको छैनः कानूनको पालना मात्रै गराएको होः प्रमुख आयुक्त थपलिया":
        return "आयोग कडा भएको छैन कानूनको पालना मात्रै गराएको हो"
    
    elif title == "विराट गोल्डकपः अफ्रिकनलाई पुनः हराउँदै संकटा सेमिफाइनलमा":
        return "विराट गोल्डकप अफ्रिकनलाई पुन हराउँदै संकटा सेमिफाइनलमा"
    
    elif title == "मौद्रिक नीतिको समीक्षामा निजी क्षेत्र: आर्थिक गतिविधि प्रभावित हुन्छ: महासंघ":
        return "मौद्रिक नीतिको समीक्षामा निजी क्षेत्रमा आर्थिक गतिविधि प्रभावित हुन्छ"
    
    elif title == "पुनः लकडाउन गर्नु परे मुलुक आर्थिक रुपमा चल्न नसक्ने हुन्छः अर्थमन्त्री":
        return "पुन लकडाउन गर्नु परे मुलुक आर्थिक रुपमा चल्न नसक्ने हुन्छ"
    
    elif title == "गन्तव्य: विद्युतीय सवारीमा सरकारी दृष्टिकोण: नीतिमा प्राथमिकता, व्यवहारमा उपेक्षा":
        return "विद्युतीय सवारीमा सरकारी दृष्टिकोण नीतिमा प्राथमिकता, व्यवहारमा उपेक्षा"
        
def predefined_slicing_imagekhabar(title, join=" ", split=" : "):

    if title == "जिया खान आत्महत्या प्रकरण : उच्च अदालतद्वारा आमा राबियाको पुन: छानबिनको निवेदन खारेज":
        return "जिया खान आत्महत्या प्रकरण उच्च अदालतद्वारा आमा राबियाको पुन छानबिनको निवेदन खारेज"
    
    elif title == "बाजा प्रतियोगितामा लूनिभाः धिमे खलः उत्कृष्ट":
        return "बाजा प्रतियोगितामा लूनिभा धिमे खल उत्कृष्ट"
    
    elif title == "केही दिनभित्रै मधेश सरकार पुनः गठन हुन्छः अध्यक्ष यादव":
        return "केही दिनभित्रै मधेश सरकार पुन गठन हुन्छ"
    
    elif title == "इरानविरुद्ध अमेरिकाले पुनः प्रतिबन्ध लगाएपछि इरानले भन्योः मनोवैज्ञानिक युद्ध किन ?":
        return "इरानविरुद्ध अमेरिकाले पुन प्रतिबन्ध लगाएपछि इरानले भन्यो मनोवैज्ञानिक युद्ध किन"
    
    elif title == "विवादास्पद विधेयक छलफलमा लैजाऔं : अधिकारी, सत्ताधारी शीर्ष नेता निरंकुश बने : देव":
        return "विवादास्पद विधेयक छलफलमा लैजाऔं, सत्ताधारी शीर्ष नेता निरंकुश बने"
    
    elif title == "महरालाई सांसदका रुपमा चिन्दैनौं : नेम्वाङ, साधारण नागरिकसरह नै छानबिन : महान्यायाधिवक्ता":
        return "महरालाई सांसदका रुपमा चिन्दैनौं"
    
    elif title == "सरकार अधिनायकवादतर्फ उन्मुख : देउवा, राज्यको ढुकुटी कार्यकर्तालाई बाँडियो : पौडेल":
        return ""
    
    elif title == "सरकार र सभामुख जनताका आवाज दबाउन उद्दत : खाँण, विपक्षीले कार्यविधि उल्लंघन गरे : रावल":
        return "सरकार र सभामुख जनताका आवाज दबाउन उद्दत, विपक्षीले कार्यविधि उल्लंघन गरे"
    
    elif title == "धर्म, संस्कृितमाथि हस्तक्षेप : सांसद केसी, गुठी विधेयकका विषयमा भ्रम छरियो : सांसद चौधरी":
        return "धर्म, संस्कृितमाथि हस्तक्षेप गुठी विधेयकका विषयमा भ्रम छरियो"
    
    elif title == "सभापति देउवा निवासमा नेवि संघको धर्नाः देउवाले भनेः ‘नयाँ निर्णय गर्छु’":
        return "सभापति देउवा निवासमा नेवि संघको धर्ना देउवाले भने ‘नयाँ निर्णय गर्छु’"
    
    elif title == "नेताहरुको विरोधमात्र नगर्नोस् : देउवा, नेतृत्वको कार्यशैलीमा समस्या : पौडेल":
        return "नेताहरुको विरोधमात्र नगर्नोस्"
    
    elif title == "रेल र पानीजहाज ल्याएरै छाडिन्छ : प्रधानमन्त्री, सरकारलाई अपदस्थ गर्न षड्यन्त्र : दाहाल":
        return "रेल र पानीजहाज ल्याएरै छाडिन्छ"
    
    elif title == "आरोप–प्रत्यारोप राम्रो होइन : श्रेष्ठ, पुनर्जीवन दिन्छ : शर्मा (भिडियो)":
        return "आरोप–प्रत्यारोप राम्रो होइन"
    
    elif title == "नीति तथा कार्यक्रम असल निबन्ध : कांग्रेस नेता कार्की, पूर्ण छैन : राजेन्द्र महतो":
        return ""
    
    elif title == "सुनको मूल्य पुनः उकालो, पुनः तोलाको ५६ हजार":
        return "सुनको मूल्य पुन उकालो तोलाको ५६ हजार"

def predefined_slicing_gorkhapatra(title, join=" ", split=" : "):
    if title == "भुसाललाई निलम्बन हुनसक्छ : बडाल, बदलाभावले कारबाही सिफारिस : भुसाल":
        return "बडाल भन्छन् भुसाललाई निलम्बन हुनसक्छ"

def predefined_slicing_ekantipur(title, join=" ", split=" : "):
    if title == "चीनमा कोरोना संक्रमण पुन: फैलिँदै : सांघाईमा ३ जनाको मृत्यु":
        return "चीनमा कोरोना संक्रमण पुन फैलिँदै सांघाईमा ३ जनाको मृत्यु"
    
    elif title == "अर्थतन्त्र सुधार्न काम गर्छु : अर्थमन्त्री, कर्मचारीको पूर्ण सहयोग रहन्छ : राजस्व सचिव":
        return "अर्थमन्त्री भन्छन् अर्थतन्त्र सुधार्न काम गर्छु राजस्व सचिव भन्छन् कर्मचारीको पूर्ण सहयोग रहन्छ"
    
def predefined_slicing_dcnepal(title, join=" ", split=" : "):

    if title == "शुक्रबार चलचित्रः हलमा ‘चि मुसी चिः इभन डेड डिजायर बिलोङ्गिङ’":
        return "शुक्रबार हलमा ‘चि मुसी चि इभन डेड डिजायर बिलोङ्गिङ’"
    
    elif title == "महोत्तरीको पुनः मतगणना परिणामः पार्टीले स्वीकार्यो, उम्मेदवारको आपत्ती":
        return "महोत्तरीको पुन मतगणना परिणाम पार्टीले स्वीकार्यो, उम्मेदवारको आपत्ती"
    
    elif title == "मुख्यमन्त्रीका रूपमा पुनः दोहोरिने मन छः मुख्यमन्त्री राई":
        return "मुख्यमन्त्री राई मुख्यमन्त्रीका रूपमा पुन दोहोरिने मन छ"
    
    elif title == "कोरोना सङ्क्रमणः भारत, पाकिस्तान र मलेसियामा क्रमशः कम हुँदै":
        return "भारत, पाकिस्तान र मलेसियामा कोरोना सङ्क्रमण कम हुँदै"
    
    elif title == "सेप्टेम्बरसम्म दक्षिणी युक्रेन पुनः कब्जा गर्छौः युक्रेन":
        return "सेप्टेम्बरसम्म दक्षिणी युक्रेन पुन कब्जा गर्छौ"
    
    elif title == "कांग्रेस महाधिवेशनः सभापति चयनका लागि पुनः मतदान दिउँसो ३ः३० बजेदेखि":
        return "कांग्रेस महाधिवेशन सभापति चयनका लागि पुन मतदान दिउँसो ३ः३० बजेदेखि"
    
    elif title == "कांग्रेस महाधिवेशन : म निर्विकल्प सभापतिको उम्मेदवार हुँ : विमलेन्द्र निधि":
        return "विमलेन्द्र निधि भन्छन् म निर्विकल्प सभापतिको उम्मेदवार हुँ"
    
    elif title == "कांग्रेस अधिवेशनः बागलुङ जिल्ला सभापतिमा ५१ प्रतिशत मत नआएपछि पुनः मतदान":
        return "बागलुङ जिल्ला सभापतिमा ५१ प्रतिशत मत नआएपछि पुन मतदान"
    
    elif title == "निकुञ्जका विस्थापित परिवारलाई पुनः स्थापना गरिनुपर्नेः मन्त्री आले":
        return "निकुञ्जका विस्थापित परिवारलाई पुन स्थापना गरिनुपर्ने"
    
    elif title == "मोतीपुर घटना : त्यो कारवाही नगरेको भए अझै ठूलो क्षति हुने थियो : प्रजिअ तिवारी":
        return "मोतीपुर घटना त्यो कारवाही नगरेको भए अझै ठूलो क्षति हुने थियो"
    
    elif title == "बालेनले कटाए २५ हजार मत: : स्थापित र सिर्जना धेरै पछि":
        return "बालेनले कटाए २५ हजार मत स्थापित र सिर्जना धेरै पछि"
    
def predefined_slicing_annapurna(title, join=" ", split=" : "):

    if title == "वार्तामा नआएपछि ईः अभियन्तासँग बालेन आक्रोशित, ईः भन्छन्- नगर प्रहरीले 'त्यसोभए मेयर भेट्दैन' भन्दै रिसाए":
        return "वार्तामा नआएपछि ई अभियन्तासँग बालेन आक्रोशित, ई भन्छन् नगर प्रहरीले 'त्यसोभए मेयर भेट्दैन' भन्दै रिसाए"
    
    elif title == "उदयपुरमा थप ८ जनामा कोरोना रिर्पोट शंकास्पद : पुन: परीक्षणका लागि धरान पठाइयो":
        return "उदयपुरमा थप ८ जनामा कोरोना रिर्पोट शंकास्पद पुन परीक्षणका लागि धरान पठाइयो"
    
    elif title == "११ घन्टापछि खुलेको पृथ्वी राजमार्ग पुनः अवरुद्धः हजारौँ यात्रु अलपत्र":
        return "११ घन्टापछि खुलेको पृथ्वी राजमार्ग पुन अवरुद्ध"
    
    elif title == "दलाई लामा अरुणाचल भ्रमणः आन्तरिक मामिलामा दखल नदेऊः भारत":
        return "दलाई लामा को अरुणाचल भ्रमण"
    
    elif title == "काश्मिरमा १३ जना मारिए, भारत भन्छ: आतंकककारी, पाकिस्तान भन्छ: सर्वसाधरण":
        return "काश्मिरमा १३ जना मारिए, भारत भन्छ आतंकककारी, पाकिस्तान भन्छ सर्वसाधरण"
    
    elif title == "सत्तापक्षीय र मधेसी मोर्चाको बैठक निष्कर्षबिहिनः पुनः छलफल गर्ने":
        return "सत्तापक्षीय र मधेसी मोर्चाको बैठक निष्कर्षबिहिन, पुन छलफल गर्ने"
    
    elif title == "नेपाल र जापानबीचको मैत्रीपूर्ण फुटबलः नेपालः १ जापानः शून्य (लाइभ)":
        return "नेपाल र जापानबीचको मैत्रीपूर्ण फुटबल नेपाल १ जापान शून्य"
    
def predefined_slicing_ekagaj(title, join=" ", split=" : "):

    if title == "एमाले भन्छ : रास्वपाले हामीलाई सघाउँछ, रास्वपा भन्छ : पुस १० को सहमतिलाई मध्यनजर गरेर निर्णय लिन्छौँ":
        return "एमाले भन्छ रास्वपाले हामीलाई सघाउँछ, रास्वपा भन्छ पुस १० को सहमतिलाई मध्यनजर गरेर निर्णय लिन्छौँ"
    
    elif title == "काँग्रेस छियाछिया भयो, पुनः जीवन दिनुपर्ने खाँचो छः डा. शशांक":
        return "काँग्रेस छियाछिया भयो, पुन जीवन दिनुपर्ने खाँचो छ"
    
    elif title == "हामी सरकारबाट हटेपछि 'समृद्ध नेपाल : सुखी नेपाली' भन्ने आँट कसैले गरेनन् : केपी ओली":
        return "हामी सरकारबाट हटेपछि 'समृद्ध नेपाल सुखी नेपाली' भन्ने आँट कसैले गरेनन्"
    
    elif title == "फुटपाथ व्यवसायी भन्छन् : गरिखान देऊ, महानगर भन्छ : हुँदैन":
        return "फुटपाथ व्यवसायी भन्छन् गरिखान देऊ महानगर भन्छ हुँदैन"
    
def predefined_slicing_nepalikhabar(title, join=" ", split=" : "):
    
    if title == "कोरोनाको उत्पत्ति स्थल पत्ता लगाउन अति आवश्यकः डब्लूएचओ, हाम्रो देशमा उत्पन्न भएको होइनः चीन":
        return "कोरोनाको उत्पत्ति स्थल पत्ता लगाउन अति आवश्यक"
    
    elif title == "कम्पनीले सम्झौता गर्दैमा नेपालमा खोप आउँदैनः मन्त्रालय, कम्पनीबारे थाहै छैनः औषधि व्यवसायी":
        return "कम्पनीले सम्झौता गर्दैमा नेपालमा खोप आउँदैन"
        
    elif title == "एमसीसी उपाध्यक्ष : तत्काल संसदबाट अनुमोदन गर्नुस्, अर्थमन्त्री : राजनीतिक सहमति खोज्दैछौँ":
        return "एमसीसी उपाध्यक्ष भन्छन् तत्काल संसदबाट अनुमोदन गर्नुस्"
    
    elif title == "अर्थमन्त्रीले भने : माथिल्लो तामाकोशी बन्द छ, आयोजना प्रमुख भन्छन् : सञ्चालनमै छ":
        return "अर्थमन्त्रीले भने माथिल्लो तामाकोशी बन्द छ, आयोजना प्रमुख भन्छन् सञ्चालनमै छ"
    
    elif title == "निषेधाज्ञाले शून्यप्रायः घरजग्गा कारोबारः साढे ८ अर्बले घट्यो राजस्व संकलन":
        return "साढे ८ अर्बले घट्यो राजस्व संकलन"
    
    elif title == "नागरिक उड्डयनमन्त्री भन्छन्ः वायुसेवा निगमले गुमेको बजार पुनः प्राप्त गर्छ":
        return "नागरिक उड्डयनमन्त्री भन्छन् वायुसेवा निगमले गुमेको बजार पुन प्राप्त गर्छ"
    
    elif title == "डिल्लीबजारमा खुल्यो १२ थरि मःमः रेरष्टुरेन्ट, कोदो, फापर, ग्रिन, चिजको मःमः पाइने":
        return "डिल्लीबजारमा खुल्यो १२ थरि मम रेरष्टुरेन्ट, कोदो, फापर, ग्रिन, चिजको मम पाइने"
    
    elif title == "सरकार भन्छः अण्डा र मासुमा आत्मनिर्भर भयौँ, तथ्यांकले देखाउँछः आयात राेकिएकाे छैन":
        return "सरकार भन्छ अण्डा र मासुमा आत्मनिर्भर भयौँ तथ्यांकले देखाउँछ आयात राेकिएकाे छैन"
    
def predefined_slicing_ujyaalo(title, join=" ", split=" : "):

    if title == "प्रदेश पाँचको बजेट परिणाममुखी छ : मुख्यमन्त्री पोखरेल, वितरणमुखी भयो : कांग्रेस":
        return "प्रदेश पाँचको बजेट परिणाममुखी छ"
    
    elif title == "सबल र सक्षम बैंक कायम हुन्छ : नेपाल, साना लगानीकर्ताहरुले कर्जा पाउन नसक्ने अवस्था आउनसक्छ : क्षेत्री":
        return "सबल र सक्षम बैंक कायम हुन्छ साना लगानीकर्ताहरुले कर्जा पाउन नसक्ने अवस्था आउनसक्छ"
    
    elif title == "आज भाइटीका : उत्तम साइत बिहान ११ : ०२ मिनेटमा":
        return "आज भाइटीका उत्तम साइत बिहान ११ ०२ मिनेटमा"
    
    elif title == "दाङ : एउटै कारमित्र मृत भेटिएका तीन बालककाे निसास्सिएर ज्यान गएकाे हाे : भिसेरा रिपाेर्ट":
        return "एउटै कारमित्र मृत भेटिएका तीन बालककाे निसास्सिएर ज्यान गएकाे हाे"
    
    elif title == "३ सय ५० भन्दा बढी कानुन संशाेधन गर्नुपर्छ : विज्ञ, छुट्टै कानुन ल्याउनुपर्छ : समिति सदस्यहरू":
        return "३ सय ५० भन्दा बढी कानुन संशाेधन गर्नुपर्छ"
    
    elif title == "सात मन्त्रीको पुनः नियुक्ति असंवैधानिक रहेको सर्वोच्चको ठहर, स्वतः पदमुक्त":
        return "सात मन्त्रीको पुन नियुक्ति असंवैधानिक रहेको सर्वोच्चको ठहर"
    
    elif title == "नेता पोख्रेलको प्रस्तावः ‘दुई अध्यक्षबीच छलफल गरेर मात्रै बैठक डाकाैँ’, प्रचण्डको जवाफः ‘एक्लाएक्लै बैठक बस्नुको अर्थ छैन’":
        return "नेता पोख्रेलको प्रस्ताव ‘दुई अध्यक्षबीच छलफल गरेर मात्रै बैठक डाकाैँ’"
    
    elif title == "कानुनी सर्वोच्चता अक्षरश: कार्यान्वयन हाेस् भनेर राजीनामा दिएको हुँ : प्याकुरेल":
        return "कार्यान्वयन हाेस् भनेर राजीनामा दिएको हुँ"
    
    elif title == "एसपीपी सम्झौता अघि बढेको छैन : नेपाली सेना, नेपालकै आग्रहमा स्वीकृत भइसक्यो : अमेरिका":
        return "एसपीपी सम्झौता अघि बढेको छैन नेपाली सेना, नेपालकै आग्रहमा स्वीकृत भइसक्यो"

def predefined_slicing_ratopati(title, join=" ", split=" : "):
    
    if title == "रंगमञ्च चरित्र : दयाहाङ र सिर्जनाको जोडी : ‘कबड्डी कबड्डी’मा ‘ट्रेलर’, ‘...र माइलो’मा ‘सिनेमा’":
        return "दयाहाङ र सिर्जनाको जोडी"
    
    elif title == "रोकिएन मुद्दाको लहरः ‘कपिल शर्मा कमेडी शो’ विरुद्ध पुनः मुद्दा दर्ता":
        return "‘कपिल शर्मा कमेडी शो’ विरुद्ध पुन मुद्दा दर्ता"
    
    elif title == "सिरियल टक : ‘उमेरमै बूढो भएर हैरान भइयो’ : सुजित थापा ‘डेभिड’, कलाकार तथा निर्देशक":
        return "‘उमेरमै बूढो भएर हैरान भइयो’"
        
    elif title == "१८ वर्षे सुहानाको पहिलो फोटोसुटः शाहरुखले भने, ‘छोरीलाई पुनः अँगालोमा बोक्दै छु’":
        return "१८ वर्षे सुहानाको पहिलो फोटोसुट"
        
    elif title == "नायिकामाथि दुष्कर्मः पुलिस पुगेपछि मिथुनको छोराको विवाह रोकियोः दुलहीको परिवार होटल छोडेर गए":
        return "पुलिस पुगेपछि मिथुनको छोराको विवाह रोकियो"
        
    elif title == "कपिल शर्माले पुनः रद्द गरे अजय देवगनसँगको सुटिङः यसपटक मामला गम्भीर":
        return "कपिल शर्माले पुन रद्द गरे अजय देवगनसँगको सुटिङ"
        
    elif title == "ट्वीटरः फिर्ता भएको केही घण्टाभित्रै पुनः सस्पेन्ड भए गायक अभिजित":
        return "ट्वीटर फिर्ता भएको केही घण्टाभित्रै पुन सस्पेन्ड भए गायक अभिजित"
        
    elif title == "अक्षय कुमार पुनः बने सबैभन्दा धनी बलिउड सेलिब्रिटीः यस वर्ष कति कमाए ?":
        return "अक्षय कुमार पुन बने सबैभन्दा धनी बलिउड सेलिब्रिटी"
        
    elif title == "‘एभेन्जर्स : एन्ड गेम’ को क्रेज : अनलाइन बुकिङ गर्ने भीडले मल्टिप्लेक्सको सर्भर डाउन":
        return "‘एभेन्जर्स एन्ड गेम’ को क्रेज अनलाइन बुकिङ गर्ने भीडले मल्टिप्लेक्सको सर्भर डाउन"
        
    elif title == "पुनः सुरु हुँदैछ द कपिल शर्मा शोः टिजर सार्वजनिक, शाहरुख खान पहिलो पाहुना":
        return "पुन सुरु हुँदैछ द कपिल शर्मा शो"
        
    elif title == "आज पुनः रेसलरसँग लड्नेछिन् राखी सावन्तः भन्छिन्, ‘पछारेको हिसाब बराबर गर्नेछु’":
        return "आज पुन रेसलरसँग लड्नेछिन् राखी सावन्त"
        
    elif title == "फ्राइडे रिलिज : देशभर ‘ऐना झ्यालको पुतली’ र ‘गोर्खा : बिनिथ द ब्रेभरी’":
        return "फ्राइडे रिलिज ‘ऐना झ्यालको पुतली’ र ‘गोर्खा बिनिथ द ब्रेभरी’"
        
    elif title == "फ्राइडे रिलिज : ‘स्पाइडर म्यान : नो वे होम’ र ‘पुष्पा’":
        return "फ्राइडे रिलिज ‘स्पाइडर म्यान नो वे होम’ र ‘पुष्पा’"
        
    elif title == "११ महिनाअघि दिवंगत पितालाई सम्झिएर भावुक भएका थिए वाजिद खानः लेखेका थिए, ‘तपाईंलाई पुनः भेट्ने प्रतिक्षा गरिरहेछु’":
        return "११ महिनाअघि दिवंगत पितालाई सम्झिएर भावुक भएका थिए वाजिद खान"
        
    elif title == "पुनः सोसल मिडियामा एक्टिभ भइन् जायरा वसीमः बताइन्, किन डिएक्टिभेट गरेकी थिइन् अकाउन्ट ?":
        return "पुन सोसल मिडियामा एक्टिभ भइन् जायरा वसीम"
        
    elif title == "श्रीमतीका लागि पुनः बाँच्न चाहन्छन् इरफान खानः भन्छन्, ‘उनकै कारण अहिलेसम्म छु’":
        return "श्रीमतीका लागि पुन बाँच्न चाहन्छन् इरफान खान भन्छन्, ‘उनकै कारण अहिलेसम्म छु’"
        
    elif title == "कोशी प्रदेश : सभामुखलाई राजीनामा गराउनु अमर्यादित खेल : एमाले":
        return "सभामुखलाई राजीनामा गराउनु अमर्यादित खेल"
    
    elif title == "पुनः समस्यामा घेरिए हाँस्यकलाकार कपिल शर्माः मुद्दा दर्ता":
        return "पुन समस्यामा घेरिए हाँस्यकलाकार कपिल शर्मा"
        

def resolve_colon_in_title(title, predefined_slicing_function):
    title_split = title.split("ः ")
    if len(title_split) == 2:
        if title_split[0].split()[-1] == "पुन":
            return title
        if len(title_split[0].split()) >= 4 and len(title_split[1].split()) >= 4:
            return ""
        elif len(title_split[0]) > len(title_split[1]): return title_split[0] + "ः"
        elif len(title_split[0]) <= len(title_split[1]): return "ः ".join(title_split)
    elif len(title_split) > 2:
        res = predefined_slicing_function(title.strip(), join="ः ", split="ः ")
        if res == None:
            print(title)
        return res
                
    title_split = title.split(": ")
    if len(title_split) == 2:
        if title_split[0].split()[-1] == "पुन":
            return title
        if len(title_split[0].split()) >= 4 and len(title_split[1].split()) >= 4:
            return ""
        elif len(title_split[0]) > len(title_split[1]): return title_split[0]
        elif len(title_split[0]) <= len(title_split[1]): return " ".join(title_split)
    elif len(title_split) > 2:
        res = predefined_slicing_function(title.strip(), split=": ")
        if res == None:
            print(title)
        return res
    
    title_split = title.split(" : ")
    if len(title_split) == 2:
        if len(title_split[0].split()) >= 4 and len(title_split[1].split()) >= 4:
            return ""
        elif len(title_split[0]) > len(title_split[1]): return title_split[0]
        elif len(title_split[0]) <= len(title_split[1]): return " ".join(title_split)
    elif len(title_split) > 2:
        res = predefined_slicing_function(title.strip(), split=" : ")
        if res == None:
            print(title)
        return res
        
    return title    

In [5]:
ratopati["title_colon_handled"] = ratopati["title"].apply(lambda x: resolve_colon_in_title(x, predefined_slicing_ratopati))

In [6]:
ujyaalo["title_colon_handled"] = ujyaalo["title"].apply(lambda x: resolve_colon_in_title(x, predefined_slicing_ujyaalo))

In [7]:
techpana["title_colon_handled"] = techpana["title"].apply(lambda x: resolve_colon_in_title(str(x), resolve_slicing))

In [8]:
nepalikhabar["title_colon_handled"] = nepalikhabar["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_nepalikhabar))

In [9]:
ekagaj["title_colon_handled"] = ekagaj["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_ekagaj))

In [10]:
annapurna["title_colon_handled"] = annapurna["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_annapurna))

In [11]:
dcnepal["title_colon_handled"] = dcnepal["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_dcnepal))

In [12]:
ekantipur["title_colon_handled"] = ekantipur["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_ekantipur))

In [13]:
gorkhapatra["title_colon_handled"] = gorkhapatra["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_gorkhapatra))

In [14]:
imagekhabar["title_colon_handled"] = imagekhabar["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_imagekhabar))

In [15]:
nagarik["title_colon_handled"] = nagarik["title"].apply(lambda x: resolve_colon_in_title(str(x), predefined_slicing_nagarik))

1. Ratopati

In [16]:
ratopati.head()

,title,news,category,published_date,title_colon_handled
0,निखिल उप्रेतीको भैरव फिल्मले अमेरिकामा रहेका न...,सनफ्रान्सिस्को-अमेरिकाका नेपालीको प्रतिक्रिया ...,मनोरञ्जन,"विहीबार, २८ माघ २०७२, ११ : ०५",निखिल उप्रेतीको भैरव फिल्मले अमेरिकामा रहेका न...
1,सुशील कोइरालाको निधनपछि चौरासी बाले खोले यस्ता...,झण्डै ४ बर्षअघि सुशील कोइरालाले प्रधानमन्त्रीक...,मनोरञ्जन,"बुधबार, २७ माघ २०७२, १५ : २५",सुशील कोइरालाको निधनपछि चौरासी बाले खोले यस्ता...
2,लिटल प्रिन्स एण्ड प्रिन्सेसको ग्रान्ड फिनाले,सुरुङ– ग्ल्यामरस नेपालले सुरुङ्गामा लिटल प्रिन...,मनोरञ्जन,"बुधबार, २७ माघ २०७२, १० : ५४",लिटल प्रिन्स एण्ड प्रिन्सेसको ग्रान्ड फिनाले
3,"अमेरिकामा सबैभन्दा धेरै कमाउने सिईओ बने पिचाई,...",अमेरिका-गूगलका सिईओ सुन्दर पिचाई अमेरिकामा सबै...,मनोरञ्जन,"मंगलबार, २६ माघ २०७२, २० : ०३","अमेरिकामा सबैभन्दा धेरै कमाउने सिईओ बने पिचाई,..."
4,"ज्योती मगरको धमाका, ‘दोहोरीमा र्याप’",काठमाडौं–‘दोहोरीमा र्याप’ बोलको गीतको भिडियो स...,मनोरञ्जन,"सोमबार, २५ माघ २०७२, १६ : ३८","ज्योती मगरको धमाका, ‘दोहोरीमा र्याप’"


Remove text inside of paranthesis

In [17]:
def remove_text_inside_paranthesis(text):
    if type(text) == float:
        return text
    return re.sub(r'\([^)]*\)', '', text)

In [18]:
# ratopati["news_no_paranthesis"] = ratopati["news"].apply(remove_text_inside_paranthesis)

In [19]:
remove_text_inside_paranthesis("hi (my)name (is)is")

'hi name is'

In [20]:
def remove_first_location_info(news):
    '''
    Takes a news article and removes the location information at the start
    '''
    # If the location is seperated by – 
    news_split = news.split("–")
    check_length = len(news_split[0].strip().split())
    if check_length == 2 or check_length == 1:
        # Only remove if the length of the possible location info contains 1 or 2 word
        news_split.pop(0)
        return " ".join(news_split)
    else:
        # If the location is seperated by । (halanta)  
        news_split = news.split("।")
        check_length = len(news_split[0].strip().split())
        if check_length == 2 or check_length == 1:
            # Only remove if the length of the possible location info contains 1 or 2 word
            news_split.pop(0)
            return " । ".join(news_split)
        else:
            # If the location is seperated by small dash -  
            news_split = news.split("-")
            check_length = len(news_split[0].strip().split())
            if check_length == 2 or check_length == 1:
                # Only remove if the length of the possible location info contains 1 or 2 word
                news_split.pop(0)
                return " ".join(news_split)
            return news

In [21]:
ratopati["news_no_location"] = ratopati["news"].apply(remove_first_location_info)

In [22]:
ratopati["news"][53000:53010]

53000    बार्सिलोनालाई हराउँदै एथ्लेटिको म्याड्रिडले स्...
53001    भारतको महाराष्ट्रमा आगामी डिसेम्बर ४ देखि ७ सम...
53002    कराते खेलका निर्णायक तथा प्रशिक्षक प्रकाशकुमार...
53003    नेपाल क्रिकेट सङ्घ (क्यान) ले बन्द प्रशिक्षणका...
53004    काठमाडौं । नेपाल क्रिकेट सङ्घ (क्यान) ले नेपाल...
53005    नेपाली क्रिकेटका ‘ग्लोबल आइकन’ सन्दीप लामिछाने...
53006    आफ्नो कारण एन्टोनी ग्रिजमान बार्सिलोनाको जीवनम...
53007    रोमेलु लुकाकुको दुई गोलको मद्दतले डेनमार्कलाई ...
53008    बङ्गलादेशमा रहेका नेपाली राष्ट्रिय फुटबल टोलीक...
53009    लुइस सुवरेजलाई कोरोनाभाइरसको संक्रमण देखिएको छ...
Name: news, dtype: object

In [23]:
# ratopati["news_no_paranthesis"][53000:53010]

In [24]:
ratopati["news_no_location"][53000:53010]

53000    बार्सिलोनालाई हराउँदै एथ्लेटिको म्याड्रिडले स्...
53001    भारतको महाराष्ट्रमा आगामी डिसेम्बर ४ देखि ७ सम...
53002    कराते खेलका निर्णायक तथा प्रशिक्षक प्रकाशकुमार...
53003    नेपाल क्रिकेट सङ्घ (क्यान) ले बन्द प्रशिक्षणका...
53004     नेपाल क्रिकेट सङ्घ (क्यान) ले नेपाल प्रिमियर ...
53005    नेपाली क्रिकेटका ‘ग्लोबल आइकन’ सन्दीप लामिछाने...
53006    आफ्नो कारण एन्टोनी ग्रिजमान बार्सिलोनाको जीवनम...
53007    रोमेलु लुकाकुको दुई गोलको मद्दतले डेनमार्कलाई ...
53008    बङ्गलादेशमा रहेका नेपाली राष्ट्रिय फुटबल टोलीक...
53009    लुइस सुवरेजलाई कोरोनाभाइरसको संक्रमण देखिएको छ...
Name: news_no_location, dtype: object

In [1]:
import string
from nltk.corpus import stopwords

In [2]:
nep_stopwrods = stopwords.words("nepali")

In [7]:
nep_stopwrods

['छ',
 'र',
 'पनि',
 'छन्',
 'लागि',
 'भएको',
 'गरेको',
 'भने',
 'गर्न',
 'गर्ने',
 'हो',
 'तथा',
 'यो',
 'रहेको',
 'उनले',
 'थियो',
 'हुने',
 'गरेका',
 'थिए',
 'गर्दै',
 'तर',
 'नै',
 'को',
 'मा',
 'हुन्',
 'भन्ने',
 'हुन',
 'गरी',
 'त',
 'हुन्छ',
 'अब',
 'के',
 'रहेका',
 'गरेर',
 'छैन',
 'दिए',
 'भए',
 'यस',
 'ले',
 'गर्नु',
 'औं',
 'सो',
 'त्यो',
 'कि',
 'जुन',
 'यी',
 'का',
 'गरि',
 'ती',
 'न',
 'छु',
 'छौं',
 'लाई',
 'नि',
 'उप',
 'अक्सर',
 'आदि',
 'कसरी',
 'क्रमशः',
 'चाले',
 'अगाडी',
 'अझै',
 'अनुसार',
 'अन्तर्गत',
 'अन्य',
 'अन्यत्र',
 'अन्यथा',
 'अरु',
 'अरुलाई',
 'अर्को',
 'अर्थात',
 'अर्थात्',
 'अलग',
 'आए',
 'आजको',
 'ओठ',
 'आत्म',
 'आफू',
 'आफूलाई',
 'आफ्नै',
 'आफ्नो',
 'आयो',
 'उदाहरण',
 'उनको',
 'उहालाई',
 'एउटै',
 'एक',
 'एकदम',
 'कतै',
 'कम से कम',
 'कसै',
 'कसैले',
 'कहाँबाट',
 'कहिलेकाहीं',
 'का',
 'किन',
 'किनभने',
 'कुनै',
 'कुरा',
 'कृपया',
 'केही',
 'कोही',
 'गए',
 'गरौं',
 'गर्छ',
 'गर्छु',
 'गर्नुपर्छ',
 'गयौ',
 'गैर',
 'चार',
 'चाहनुहुन्छ',
 'चाहन्छु',
 'चाहिए

In [3]:
def remove_emojis_english_and_numbers(data):
    '''
    Removes emojis, non-nepali texts and numbers from the given text
    '''
    # Removes emoji from given data
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    res = re.sub(emoj, '', data)
    res = re.sub('[0-9]+', '', res)
    return re.sub('[a-zA-Z]', '', res)

In [6]:
def preprocess_text(data, is_title=False):
    if type(data) == float:
        return data
    data = data.replace("-", " ").replace("—", " ").replace("‘", " ").replace("’", " ").replace("।", " ").replace("–", " ").replace("“", " ").replace("”", " ") .replace("\n", " ").replace("–", " ").replace("ः", "").replace(":", "").replace("ः ", " ").replace(": ", " ")
    no_extra_spaces = " ".join(data.split())
    no_emoji_english_numbers = remove_emojis_english_and_numbers(no_extra_spaces)
    no_punc = "".join([char for char in no_emoji_english_numbers if char not in (string.punctuation)])
    extra = " ".join(no_punc.split())
    # Remove stopwords from news only
    if not is_title:
        no_stopwords = [word for word in extra.split() if word.strip() not in nep_stopwrods]
        return " ".join(no_stopwords)
    else:
        return extra
print("विप्लवको क्रान्ति ! खसी नदिएपछि हत्या गरेका थिए शिक्षक श्रेष्ठलाई")
preprocess_text("विप्लवको क्रान्ति ! खसी नदिएपछि हत्या गरेका थिए शिक्षक श्रेष्ठलाई")
# print(ratopati["title"][4345])
# preprocess_text(ratopati["title"][4345], True)

विप्लवको क्रान्ति ! खसी नदिएपछि हत्या गरेका थिए शिक्षक श्रेष्ठलाई


'विप्लवको क्रान्ति खसी नदिएपछि हत्या शिक्षक श्रेष्ठलाई'

In [29]:
# ratopati["news_cleaned"] = ratopati["news_no_location"].apply(preprocess_text)

In [30]:
# ratopati["news_cleaned"]

In [31]:
ratopati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56680 entries, 0 to 56679
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                56680 non-null  object
 1   news                 56680 non-null  object
 2   category             56680 non-null  object
 3   published_date       56680 non-null  object
 4   title_colon_handled  56680 non-null  object
 5   news_no_location     56680 non-null  object
dtypes: object(6)
memory usage: 2.6+ MB


Repeat same for titles except for location removal

In [32]:
# ratopati["title_no_paranthesis"] = ratopati["title"].apply(remove_text_inside_paranthesis)

In [33]:
# ratopati["title_cleaned"] = ratopati["title_no_paranthesis"].apply(lambda x: preprocess_text(x, True))

In [34]:
# ratopati["title_cleaned"][4905]

In [35]:
# ratopati["title"] = ratopati["title_cleaned"].apply(lambda x: x)
# ratopati["news"] = ratopati["news_cleaned"].apply(lambda x: x)

In [36]:
# ratopati.to_csv('../news_clean/ratopati_cleaned.csv', columns=["title", "news", "category"], index=False)

Remove the location info from all news articles seperatedly since they may be in different form

2. Ujjyalo

In [37]:
def remove_first_location_info_ujyaalo(news):
    '''
    Takes a news article and removes the location information at the start
    '''
    # If the location is seperated by – 
    news_split = news.split("–")
    check_length = len(news_split[0].strip().split())
    if check_length == 2 or check_length == 1:
        # Only remove if the length of the possible location info contains 1 or 2 word
        news_split.pop(0)
        return " ".join(news_split)
    else:
        # If the location is seperated by small dash -  
        news_split = news.split("-")
        check_length = len(news_split[0].strip().split())
        if check_length == 2 or check_length == 1:
            # Only remove if the length of the possible location info contains 1 or 2 word
            news_split.pop(0)
            return " ".join(news_split)
        return news
ujyaalo["news_no_location"] = ujyaalo["news"].apply(remove_first_location_info_ujyaalo)

In [38]:
len(ujyaalo["news"])

23884

In [39]:
ujyaalo["news"][22000:22010]

22000    काठमाडौं - नेपाल कम्युनिष्ट पार्टी (नेकपा) का ...
22001    काठमाण्डौ - कोरोना सङ्क्रमणको परीक्षणका लागि स...
22002    काठमाण्डौ - आउँदो आर्थिक वर्षको लागि सरकारको न...
22003    काठमाण्डौ - नेपालमा कोरोना भाइरसको सङ्क्रमण बढ...
22004    काठमाण्डौ - सत्तारुढ नेपाल कम्युनिस्ट पार्टी (...
22005    काठमाण्डौ - नेपाली कांग्रेसले कोरोना संक्रमणपछ...
22006    काठमाण्डौ – नेपाल कम्युनिस्ट पार्टी (नेकपा) को...
22007    काठमाण्डौ - गृहमन्त्री रामबहादुर थापाले लकडाउन...
22008    काठमाण्डौ - संघीय संसदका महासचिव डा.भरतराज गौत...
22009    काठमाण्डाै – कोरोना रोकथाम तथा नियन्त्रण उच्चस...
Name: news, dtype: object

In [40]:
ujyaalo["news_no_location"][22000:22010]

22000     नेपाल कम्युनिष्ट पार्टी (नेकपा) का २० जना स्थ...
22001     कोरोना सङ्क्रमणको परीक्षणका लागि संघीय संसद अ...
22002     आउँदो आर्थिक वर्षको लागि सरकारको नीति तथा कार...
22003     नेपालमा कोरोना भाइरसको सङ्क्रमण बढेसँगै सबै क...
22004     सत्तारुढ नेपाल कम्युनिस्ट पार्टी (नेकपा) भित्...
22005     नेपाली कांग्रेसले कोरोना संक्रमणपछिको जटिल अव...
22006     नेपाल कम्युनिस्ट पार्टी (नेकपा) को सचिवालय बै...
22007     गृहमन्त्री रामबहादुर थापाले लकडाउन जारी रहे ज...
22008     संघीय संसदका महासचिव डा.भरतराज गौतमले पद तथा ...
22009     कोरोना रोकथाम तथा नियन्त्रण उच्चस्तरीय समन्वय...
Name: news_no_location, dtype: object

3. Techpana

In [41]:
def remove_first_location_info_techpana(news):
    '''
    Takes a news article and removes the location information at the start
    '''
    # If the location is seperated by । (halanta) 
    news_split = news.split("।")
    check_length = len(news_split[0].strip().split())
    if check_length == 2 or check_length == 1:
        # Only remove if the length of the possible location info contains 1 or 2 word
        news_split.pop(0)
        check_length = len(news_split[0].strip().split())
        if check_length <= 3:
            news_split.pop(0)
        return " । ".join(news_split)
    return news
techpana["news_no_location"] = techpana["news"].apply(remove_first_location_info_techpana)

In [42]:
len(techpana["news"])

21995

In [43]:
techpana["news"][21000:21010]

21000    काठमाडौं । एप्पलको एप स्टोरमा अनौपचारिक रुपमा ...
21001    काठमाडौं । सिम डुब्लिकेशनगरी मोबाइल बैंकिङबाट ...
21002    काठमाडौं । स्थापनाको पहिलो वर्षमै माइपे डिजिटल...
21003    काठमाडौं । विद्युतीय कारोबारलाई प्रबर्द्धन गर्...
21004    काठमाडौं । ट्विटर, गुगल, टाइगर ग्लोबल र टेमासे...
21005    काठमाडौं । हामीले गाउँ घर तिर गुईठा (गोबरबाट ब...
21006    काठमाडौं । आइएमई पेले मोबाईल रिचार्जमा ५ प्रति...
21007    काठमाडौं । एप्पलले पछिल्लो केही समयदेखि नयाँ म...
21008    काठमाडौं । नवनियुक्त सञ्चार तथा सूचना प्रविधि ...
21009    काठमाडौं । नयाँ ऊर्जा सवारी साधन (एनईभी) को ची...
Name: news, dtype: object

In [44]:
techpana["news_no_location"][21000:21010]

21000     एप्पलको एप स्टोरमा अनौपचारिक रुपमा च्याटजीपीट...
21001     सिम डुब्लिकेशनगरी मोबाइल बैंकिङबाट रकम ठगी गर...
21002     स्थापनाको पहिलो वर्षमै माइपे डिजिटल वालेटले प...
21003     विद्युतीय कारोबारलाई प्रबर्द्धन गर्ने उद्देश्...
21004     ट्विटर, गुगल, टाइगर ग्लोबल र टेमासेकद्वारा सम...
21005     हामीले गाउँ घर तिर गुईठा (गोबरबाट बनाइएको दाउ...
21006     आइएमई पेले मोबाईल रिचार्जमा ५ प्रतिशतसम्मको क...
21007     एप्पलले पछिल्लो केही समयदेखि नयाँ म्याकबुक प्...
21008     नवनियुक्त सञ्चार तथा सूचना प्रविधि मन्त्री रे...
21009     नयाँ ऊर्जा सवारी साधन (एनईभी) को चीनको सबैभन्...
Name: news_no_location, dtype: object

4. Ekagaz

In [45]:
def remove_first_location_info_ekagaz(news):
    '''
    Takes a news article and removes the location information at the start
    '''
    # If the location is seperated by – 
    news_split = news.split("–")
    check_length = len(news_split[0].strip().split())
    if check_length == 2 or check_length == 1:
        # Only remove if the length of the possible location info contains 1 or 2 word
        news_split.pop(0)
        return " ".join(news_split)
    else:
        # If the location is seperated by small dash -  
        news_split = news.split("-")
        check_length = len(news_split[0].strip().split())
        if check_length == 2 or check_length == 1:
            # Only remove if the length of the possible location info contains 1 or 2 word
            news_split.pop(0)
            return " ".join(news_split)
        else: 
            # If the location is seperated by ।           
            news_split = news.split("।")
            check_length = len(news_split[0].strip().split())
            if check_length == 2 or check_length == 1:
                # Only remove if the length of the possible location info contains 1 or 2 word
                news_split.pop(0)
                return " । ".join(news_split)
        return news
ekagaj["news_no_location"] = ekagaj["news"].apply(remove_first_location_info_ekagaz)

In [46]:
len(ekagaj["news"])

18270

In [47]:
ekagaj["news"][7000:7010]

7000    काठमाडौँ– राष्ट्रिय निर्वाचन आयोगले समानुपातिक...
7001    काठमाडौँ- प्रतिनिधिसभाबाट पारित भएको विनियोजन ...
7002    काठमाडौँ - प्रधानमन्त्री पुष्पकमल दाहाल ‘प्रचण...
7003    काठमाडौँ- राष्ट्रियसभाबाट ‘नेपाल विश्वविद्यालय...
7004    काठमाडौँ - राष्ट्रियसभाका सांसद खिमलाल देवकोटा...
7005    काठमाडौँ- कृषि, सहकारी तथा प्राकृतिक स्रोत समि...
7006    काठमाडौं - दलितका लागि छुट्टै बजेटको व्यवस्था ...
7007    काठमाडौं - नेकपा एकीकृत समाजवादीका अध्यक्ष माध...
7008    काठमाडौँ- राष्ट्रिय सभा उपाध्यक्ष उर्मिला अर्य...
7009    कैलाली- प्रधानमन्त्री पुष्पकमल दाहाल प्रचण्ड आ...
Name: news, dtype: object

In [48]:
ekagaj["news_no_location"][7000:7010]

7000     राष्ट्रिय निर्वाचन आयोगले समानुपातिक सांसद दु...
7001     प्रतिनिधिसभाबाट पारित भएको विनियोजन विधेयक, २...
7002     प्रधानमन्त्री पुष्पकमल दाहाल ‘प्रचण्ड’ले सरका...
7003     राष्ट्रियसभाबाट ‘नेपाल विश्वविद्यालय विधेयक–२...
7004     राष्ट्रियसभाका सांसद खिमलाल देवकोटाले नेपालमा...
7005     कृषि, सहकारी तथा प्राकृतिक स्रोत समितिको आन्त...
7006     दलितका लागि छुट्टै बजेटको व्यवस्था गर्न माग ग...
7007     नेकपा एकीकृत समाजवादीका अध्यक्ष माधवकुमार नेप...
7008     राष्ट्रिय सभा उपाध्यक्ष उर्मिला अर्यालले दलित...
7009     प्रधानमन्त्री पुष्पकमल दाहाल प्रचण्ड आगामी से...
Name: news_no_location, dtype: object

5. Annapurna

In [49]:
def remove_first_location_info_annapurna(news):
    '''
    Takes a news article and removes the location information at the start
    '''

    # If the location is seperated by ः
    news_split = news.split("ः")
    check_length = len(news_split[0].strip().split())
    if check_length == 2 or check_length == 1:
        # Only remove if the length of the possible location info contains 1 or 2 word
        news_split.pop(0)
        return " ".join(news_split)
    else:
        # If the location is seperated by :  
        news_split = news.split(":")
        check_length = len(news_split[0].strip().split())
        if check_length == 2 or check_length == 1:
            # Only remove if the length of the possible location info contains 1 or 2 word
            news_split.pop(0)
            return " ".join(news_split)
        return news
annapurna["news_no_location"] = annapurna["news"].apply(remove_first_location_info_annapurna)

In [50]:
annapurna["news"][7000:7010]

7000    मुम्बई : हट र आकर्षक हुँदाहुँदै पनि उर्वशी रौट...
7001    प्रायः नाटकका संवाद, पृष्ठभूमिका संगीत र कथाका...
7002    काठमाडौं : मिस नेपाल शृंखला खतिवडा नेपाल र फ्र...
7003    नयाँदिल्ली : बलिउडका नवविवाहित कलाकार जोडी रणव...
7004    अमेरिका : तेस्रो ग्लोबल नेपाली फिल्म अवार्ड २०...
7005    काठमाडौं : सुबिन भट्टराईको उपन्यास समर लभमा आध...
7006    नर्सरीदेखि कक्षा १० सम्म म एउटै विद्यालयमा पढे...
7007    मुम्बई : बलिउड अभिनेता अक्षय कुमारले २०१९ मा ठ...
7008    काठमाडौं : नेपाली फिल्म निर्माण सुरु भएको ५३ व...
7009    काठमाडौं : फिल्म ‘हृदयभरि’ निर्माण हुने भएको छ...
Name: news, dtype: object

In [51]:
annapurna["news_no_location"][7000:7010]

7000     हट र आकर्षक हुँदाहुँदै पनि उर्वशी रौटेलाले बल...
7001     नाटकका संवाद, पृष्ठभूमिका संगीत र कथाका दृश्य...
7002     मिस नेपाल शृंखला खतिवडा नेपाल र फ्रान्सको सम्...
7003     बलिउडका नवविवाहित कलाकार जोडी रणवीर सिंह अभिन...
7004     तेस्रो ग्लोबल नेपाली फिल्म अवार्ड २०१८ सम्पन्...
7005     सुबिन भट्टराईको उपन्यास समर लभमा आधारित नेपाल...
7006    नर्सरीदेखि कक्षा १० सम्म म एउटै विद्यालयमा पढे...
7007     बलिउड अभिनेता अक्षय कुमारले २०१९ मा ठूला फिल्...
7008     नेपाली फिल्म निर्माण सुरु भएको ५३ वर्ष पुग्दा...
7009     फिल्म ‘हृदयभरि’ निर्माण हुने भएको छ । सुरज सु...
Name: news_no_location, dtype: object

In [52]:
def remove_first_location_info_dcnepal(news):
    '''
    Takes a news article and removes the location information at the start
    '''

    # If the location is seperated by ।
    news_split = news.split("।")
    check_length = len(news_split[0].strip().split())
    if check_length == 2 or check_length == 1:
        # Only remove if the length of the possible location info contains 1 or 2 word
        news_split.pop(0)
        return " ".join(news_split)
    else:
        # If the location is seperated by , 
        news_split = news.split(",")
        check_length = len(news_split[0].strip().split())
        if check_length == 2 or check_length == 1:
            # Only remove if the length of the possible location info contains 1 or 2 word
            news_split.pop(0)
            return " ".join(news_split)
        return news
dcnepal["news_no_location"] = dcnepal["news"].apply(remove_first_location_info_dcnepal)

In [53]:
dcnepal["news"][20:30]

20    काठमाडौं । त्रिभुवन विश्वविद्यालयका उपकुलपति प...
21    काठमाडौं । नेकपा (एमाले) का अध्यक्ष केपी शर्मा...
22    विराटनगर । कोशी प्रदेशमा मन्त्रिपरिषद विस्तार ...
23    काठमाडौं । प्रधानमन्त्री पुष्पकमल दाहाल प्रचण्...
24    काठमाडौं। प्रधानमन्त्री पुष्पकमल दाहाल ‘प्रचण्...
25    काठमाडौं। प्रतिनिधि सभाको राज्य व्यवस्था तथा स...
26    काठमाडौं। प्यालेस्टाइनी अतिबादी समूह हमासको कब...
27    काठमाडौं। सरकारको सिफारिसमा कैद कटीसहित आम माफ...
28    काठमाडौं । युवा तथा खेलकुदमन्त्री डिगबहादुर लि...
29    काठमाडौं । प्रतिनिधिसभाको चालु अधिवेशन आज राति...
Name: news, dtype: object

In [54]:
dcnepal["news_no_location"][20:30]

20     त्रिभुवन विश्वविद्यालयका उपकुलपति प्रा डा धर्...
21     नेकपा (एमाले) का अध्यक्ष केपी शर्मा ओलीले नेप...
22     कोशी प्रदेशमा मन्त्रिपरिषद विस्तार भएको छ   क...
23     प्रधानमन्त्री पुष्पकमल दाहाल प्रचण्डको राजीना...
24     प्रधानमन्त्री पुष्पकमल दाहाल ‘प्रचण्ड’ ले जलव...
25     प्रतिनिधि सभाको राज्य व्यवस्था तथा सुशासन समि...
26     प्यालेस्टाइनी अतिबादी समूह हमासको कब्जामा रहे...
27     सरकारको सिफारिसमा कैद कटीसहित आम माफी पाएका र...
28     युवा तथा खेलकुदमन्त्री डिगबहादुर लिम्बूसँग ने...
29     प्रतिनिधिसभाको चालु अधिवेशन आज राति १२ः०० बजे...
Name: news_no_location, dtype: object

In [55]:
def remove_first_location_info_ekantipur(news):
    '''
    Takes a news article and removes the location information at the start
    '''

    # If the location is seperated by – 
    news_split = news.split("—")
    check_length = len(news_split[0].strip().split())
    if check_length == 2 or check_length == 1:
        # Only remove if the length of the possible location info contains 1 or 2 word
        news_split.pop(0)
        return " ".join(news_split)
    else:
        # If the location is seperated by small dash -  
        news_split = news.split("-")
        check_length = len(news_split[0].strip().split())
        if check_length == 2 or check_length == 1:
            # Only remove if the length of the possible location info contains 1 or 2 word
            news_split.pop(0)
            return " ".join(news_split)
        return news
ekantipur["news_no_location"] = ekantipur["news"].apply(remove_first_location_info_ekantipur)

In [56]:
ekantipur["news"][20:30]

20    काठमाडौँ — भक्तराज आचार्य संगीत प्रतिष्ठानले ‘...
21    तनहुँ — मुलुकमा राजनीतिक परिवर्तनका लागि गुरुङ...
22    काठमाडौँ — अभिनेत्री कंगना रनावत छिन् र बेलाबख...
23    काठमाडौँ — मिस नेपाल ग्रान्ड फिनालेको मञ्च व्य...
24    काठमाडौँ — गलामा जरिजडित दुबोको माला, भव्य आसन...
25    काठमाडौं — कीर्तिपुरमा बिहीबार राति अबेरसम्म च...
26    बलिउड अभिनेक्री सोनाली बेन्द्रे हालै मात्र क्य...
27    काठमाडौँ — लामो समय दक्षिण भारतीय फिल्मले कब्ज...
28    काठमाडौँ — पृष्ठभूमिमा भिन्न परिवेश, विभिन्न र...
29    काठमाडौँ — बलिउडदेखि हलिउडसम्म आफ्नो फिल्मी कर...
Name: news, dtype: object

In [57]:
ekantipur["news_no_location"][20:30]

20     भक्तराज आचार्य संगीत प्रतिष्ठानले ‘भक्तराज आच...
21     मुलुकमा राजनीतिक परिवर्तनका लागि गुरुङ समुदाय...
22     अभिनेत्री कंगना रनावत छिन् र बेलाबखत बलिउड गर...
23     मिस नेपाल ग्रान्ड फिनालेको मञ्च व्यवस्थापनमा ...
24     गलामा जरिजडित दुबोको माला, भव्य आसन । सय वर्ष...
25     कीर्तिपुरमा बिहीबार राति अबेरसम्म चलेको ग्रान...
26    बलिउड अभिनेक्री सोनाली बेन्द्रे हालै मात्र क्य...
27     लामो समय दक्षिण भारतीय फिल्मले कब्जा जमाएको य...
28     पृष्ठभूमिमा भिन्न परिवेश, विभिन्न रङ, थुप्रै ...
29     बलिउडदेखि हलिउडसम्म आफ्नो फिल्मी करिअरलाई विस...
Name: news_no_location, dtype: object

In [58]:
def remove_first_location_info_imagekhabar(news):
    '''
    Takes a news article and removes the location information at the start
    '''

    # If the location is seperated by । 
    news_split = news.split("।")
    check_length = len(news_split[0].strip().split())
    if check_length <= 8:
        # Only remove if the length of the possible location info contains less than 8 words
        news_split.pop(0)
        return " ".join(news_split)
    return news
imagekhabar["news_no_location"] = imagekhabar["news"].apply(remove_first_location_info_imagekhabar)

In [59]:
imagekhabar["news"][42150:42160]

42150    सिडिएससी, नेपाल स्टक एक्सचेन्ज लि.र उद्योग वाण...
42151    यो योजनाअन्तर्गत परीक्षार्थीले मात्र रु १ मा ए...
42152    आइतबार नेप्से २४ अंकले झरेर ११९६ मा आएर थामियो...
42153    सुनौलीस्थित बेलहिया नाकासँग सीमाना जोडिएको भार...
42154    मलेसियन कम्पनी वाइबिडी सिमेक्स कम्पनी र महानगर...
42155    विभागले रुट इजाजत दिने यातायात व्यवस्थापन कार्...
42156    चालू आर्थिक वर्षको आठ महिनामा स्वदेशी तथा विदे...
42157    तेह्रथुम चैत १८ । तेह्रथुममा साना तथा घरेलु व्...
42158    शनिबार राजधानीमा सम्पन्न महाधिवेशनले विनोद श्र...
42159    पछिल्लो समय स्याङगाउँका अधिकांश किसान व्यावसाय...
Name: news, dtype: object

In [60]:
imagekhabar["news_no_location"][42150:42160]

42150    सिडिएससी, नेपाल स्टक एक्सचेन्ज लि.र उद्योग वाण...
42151    यो योजनाअन्तर्गत परीक्षार्थीले मात्र रु १ मा ए...
42152    आइतबार नेप्से २४ अंकले झरेर ११९६ मा आएर थामियो...
42153    सुनौलीस्थित बेलहिया नाकासँग सीमाना जोडिएको भार...
42154    मलेसियन कम्पनी वाइबिडी सिमेक्स कम्पनी र महानगर...
42155    विभागले रुट इजाजत दिने यातायात व्यवस्थापन कार्...
42156    चालू आर्थिक वर्षको आठ महिनामा स्वदेशी तथा विदे...
42157     तेह्रथुममा साना तथा घरेलु व्यवसायमा ३ हजार ३ ...
42158    शनिबार राजधानीमा सम्पन्न महाधिवेशनले विनोद श्र...
42159    पछिल्लो समय स्याङगाउँका अधिकांश किसान व्यावसाय...
Name: news_no_location, dtype: object

Combine all the processes into a single function

In [61]:
def preprocess(df, filename, no_location=False):
    if no_location:
        df["news_no_paranthesis"] = df["news"].apply(remove_text_inside_paranthesis)
    else:   
        df["news_no_paranthesis"] = df["news_no_location"].apply(remove_text_inside_paranthesis)
    df["news_cleaned"] = df["news_no_paranthesis"].apply(preprocess_text)
    df["title_no_paranthesis"] = df["title_colon_handled"].apply(remove_text_inside_paranthesis)
    df["title_cleaned"] = df["title_no_paranthesis"].apply(lambda x: preprocess_text(x, True))
    df["title"] = df["title_cleaned"].apply(lambda x: x)
    df["news"] = df["news_cleaned"].apply(lambda x: x)
    df.to_csv(f'../news_clean/{filename}.csv', columns=["title", "news", "category"], index=False)

In [62]:
preprocess(ratopati, 'ratopati_cleaned')

In [63]:
preprocess(ujyaalo, 'ujyaalo_cleaned')

In [64]:
preprocess(techpana, 'techpana_cleaned')

In [65]:
preprocess(nepalikhabar, 'nepalikhabar_cleaned', no_location=True)

In [66]:
preprocess(ekagaj, 'ekagaj_cleaned')

In [67]:
preprocess(annapurna, 'annapurna_cleaned')

In [68]:
preprocess(dcnepal, 'dcnepal_cleaned')

In [69]:
preprocess(ekantipur, 'ekantipur_cleaned')

In [70]:
preprocess(gorkhapatra, 'gorkhapatra_cleaned', no_location=True)

In [71]:
preprocess(imagekhabar, 'imagekhabar_cleaned')

In [72]:
preprocess(nagarik, 'nagarik_cleaned', no_location=True)

In [73]:
ratopati_cleaned = pd.read_csv('../news_clean/ratopati_cleaned.csv')
ujyaalo_cleaned = pd.read_csv('../news_clean/ujyaalo_cleaned.csv')
techpana_cleaned = pd.read_csv('../news_clean/techpana_cleaned.csv')
nepalikhabar_cleaned = pd.read_csv('../news_clean/nepalikhabar_cleaned.csv')
ekagaj_cleaned = pd.read_csv('../news_clean/ekagaj_cleaned.csv')
annapurna_cleaned = pd.read_csv('../news_clean/annapurna_cleaned.csv')
dcnepal_cleaned = pd.read_csv('../news_clean/dcnepal_cleaned.csv')
ekantipur_cleaned = pd.read_csv('../news_clean/ekantipur_cleaned.csv')
gorkhapatra_cleaned = pd.read_csv('../news_clean/gorkhapatra_cleaned.csv')
imagekhabar_cleaned = pd.read_csv('../news_clean/imagekhabar_cleaned.csv')
nagarik_cleaned = pd.read_csv('../news_clean/nagarik_cleaned.csv')

In [74]:
print(len(ratopati_cleaned))
print(len(ujyaalo_cleaned))
print(len(techpana_cleaned))
print(len(nepalikhabar_cleaned))
print(len(ekagaj_cleaned))
print(len(annapurna_cleaned))
print(len(dcnepal_cleaned))
print(len(ekantipur_cleaned))
print(len(gorkhapatra_cleaned))
print(len(imagekhabar_cleaned))
print(len(nagarik_cleaned))

56680
23884
21995
18264
18270
50797
42111
19430
23350
45047
51812


In [75]:
print(len(ratopati))
print(len(ujyaalo))
print(len(techpana))
print(len(nepalikhabar))
print(len(ekagaj))
print(len(annapurna))
print(len(dcnepal))
print(len(ekantipur))
print(len(gorkhapatra))
print(len(imagekhabar))
print(len(nagarik))

56680
23884
21995
18264
18270
50797
42111
19430
23350
45047
51812


In [76]:
cleaned = pd.concat([ratopati_cleaned, ujyaalo_cleaned, techpana_cleaned, nepalikhabar_cleaned, ekagaj_cleaned, annapurna_cleaned, dcnepal_cleaned, ekantipur_cleaned, gorkhapatra_cleaned, imagekhabar_cleaned, nagarik_cleaned], axis=0)

In [77]:
len(cleaned["title"])

371640

In [78]:
cleaned["category"].value_counts()

category
राजनीति              59975
देश/प्रदेश           51799
समाज                 50207
अर्थ / वाणिज्य       43020
खेलकुद               42367
विश्व                37817
मनोरञ्जन             34178
विज्ञान र प्रविधि    23130
स्वास्थ्य            22392
शिक्षा                6755
Name: count, dtype: int64

In [79]:
cleaned.dropna(axis=0, inplace=True)

In [80]:
len(cleaned["news"])

362897

In [81]:
cleaned.to_csv('../news_clean/full_cleaned.csv', index=False)

In [82]:
cleaned.head(5)

,title,news,category
0,निखिल उप्रेतीको भैरव फिल्मले अमेरिकामा रहेका न...,अमेरिकाका नेपालीको प्रतिक्रिया लिने इच्छा व्यक...,मनोरञ्जन
1,सुशील कोइरालाको निधनपछि चौरासी बाले खोले यस्ता...,झण्डै ४ बर्षअघि सुशील कोइरालाले प्रधानमन्त्रीक...,मनोरञ्जन
2,लिटल प्रिन्स एण्ड प्रिन्सेसको ग्रान्ड फिनाले,ग्ल्यामरस नेपालले सुरुङ्गामा लिटल प्रिन्स एन्ड...,मनोरञ्जन
3,अमेरिकामा सबैभन्दा धेरै कमाउने सिईओ बने पिचाई ...,गूगलका सिईओ सुन्दर पिचाई अमेरिकामा सबैभन्दा कम...,मनोरञ्जन
4,ज्योती मगरको धमाका दोहोरीमा र्याप,दोहोरीमा र्याप बोलको गीतको भिडियो सार्वजनिक चर...,मनोरञ्जन


In [83]:
df = pd.read_csv('../news_clean/full_cleaned.csv')
len(df["news"])

362897